In [ ]:
from mxnet import gluon
from mxnet.gluon.data.vision import datasets, transforms

In [ ]:
train_dataset = datasets.MNIST(train=True).transform_first(transforms.ToTensor())

mx_train_data = gluon.data.DataLoader(
    train_dataset, batch_size=100, shuffle=True)

In [ ]:
import mxnet.gluon.nn as mx_nn

mx_net = mx_nn.Sequential()
mx_net.add(
    mx_nn.Conv2D(channels = 6, kernel_size = 5, strides = 1, 
                activation = 'tanh'),
    mx_nn.AvgPool2D(pool_size = 2, strides = 2),
    
    mx_nn.Conv2D(channels = 16, kernel_size = 5, strides = 1, 
                activation = 'tanh'),
    mx_nn.AvgPool2D(pool_size = 2, strides = 2),
    
    #Dense has flatten = True by default
    mx_nn.Dense(units = 120, activation = 'tanh'),
    mx_nn.Dense(units = 84, activation = 'tanh'),
    mx_nn.Dense(units = 10, flatten = False)
)

mx_net.initialize()

In [ ]:
mx_loss_fn = gluon.loss.SoftmaxCrossEntropyLoss()
mx_trainer = gluon.Trainer(mx_net.collect_params(),
                           'sgd', {'learning_rate': 0.1})

In [ ]:
import time
from mxnet import autograd

for epoch in range(5):
    total_loss = .0
    tic = time.time()
    for X, y in mx_train_data:
        with autograd.record():
            loss = mx_loss_fn(mx_net(X), y)
        loss.backward()
        mx_trainer.step(batch_size=100)
        total_loss += loss.mean().asscalar()
    print('epoch %d, avg loss %.4f, time %.2f' % (
        epoch, total_loss/len(mx_train_data), time.time()-tic))